In [1]:
import pandas as pd
import os
import numpy as np
import time
from multiprocessing.dummy import Pool as ThreadPool
import datetime
import pickle

In [2]:
calculated_features = r"Y:\CIS-PD Study\MJFF Curation\all pre-visit calc features"
watch_accelerometer = r"Y:\CIS-PD Study\MJFF Curation\all pre-visit ACC"
combined_data = r"Y:\\CIS-PD Study\\MJFF Curation\\combined pre-visit data"

In [3]:
files = os.listdir(calculated_features)
files.insert(0, files.pop(files.index('142620 2017-10-30.csv')))
files.insert(0, files.pop(files.index('142616 2017-10-03.csv')))
files.insert(0, files.pop(files.index('142557 2017-06-29.csv')))
files.insert(0, files.pop(files.index('142616 2018-02-05.csv')))
files.insert(0, files.pop(files.index('142578 2017-09-18.csv')))
print(files)

['142578 2017-09-18.csv', '142616 2018-02-05.csv', '142557 2017-06-29.csv', '142616 2017-10-03.csv', '142620 2017-10-30.csv', '142568 2017-09-18.csv', '142558 2017-07-14.csv', '142609 2017-09-01.csv', '142612 2017-09-06.csv', '142568 2018-02-28.csv', '142615 2017-10-05.csv', '142620 2017-08-10.csv', '142593 2017-11-06.csv', '142601 2017-07-13.csv', '142603 2017-10-13.csv', '142615 2017-07-21.csv', '142558 2017-09-21.csv', '142595 2017-10-12.csv', '142621 2017-10-23.csv', '142608 2017-11-09.csv', '142580 2017-08-01.csv', '142577 2017-11-30.csv', '142603 2017-08-07.csv', '142575 2017-08-29.csv', '142608 2017-09-11.csv', '142619 2017-08-10.csv', '142577 2017-10-11.csv', '142570 2017-07-13.csv', '142604 2017-10-02.csv', '142623 2017-08-03.csv', '142585 2017-09-20.csv', '142583 2017-10-13.csv', '142616 2017-08-01.csv', '142623 2017-10-12.csv', '142604 2017-08-14.csv', '142566 2017-08-09.csv', '142561 2017-07-03.csv', '142619 2017-07-21.csv', '142605 2017-10-11.csv', '142559 2017-07-20.csv',

In [4]:
len(files)

124

In [5]:
def drop_Movements_Level(file):
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file)
    for index, row in cf.iterrows():
        if (row['measurement_id'] == 11):
            cf = cf.drop(index)
    cf.to_csv(calculated_features + "\\" + file)

In [6]:
def drop_extra_index_columns(file):
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file)
    cols = list(cf.columns.values)
    for col in cols:
        if "Unnamed" in col:
            cf = cf.drop(col, axis = 1)
    cf.to_csv(calculated_features + "\\" + file)

In [7]:
def find_corresponding_cf_data(tm, cf):
    tm = tm - datetime.timedelta(minutes=0,
                             seconds=tm.second % 5,
                             microseconds=tm.microsecond)
    tm = tm.strftime('%Y-%m-%d %H:%M:%S.000')
#     print(cf.index[0], tm,cf.index[0][1]==tm, cf.index[0]==(10,tm))
    return [cf.loc[(13,tm)].value,cf.loc[(10,tm)].value,cf.loc[(7,tm)].value]
                
                

In [14]:
def merge_file_2(file):
    t1 = time.time()
    print("2:")
    print(file)
    cf = cf = pd.read_csv(calculated_features + "\\" + file)
    wa = pd.read_csv(watch_accelerometer + "\\" + file)
    
    #if os.path.isfile(combined_data + "\\" + file):
    #    return
    
    wa['timestamp'] = pd.to_datetime(wa['timestamp'])
    cf['timestamp'] = pd.to_datetime(cf['timestamp'])
    cf.index = pd.MultiIndex.from_tuples([(row[1].measurement_id,row[1].timestamp) for row in cf.iterrows()])
        
    df = wa['timestamp'].apply(lambda tm: pd.Series(find_corresponding_cf_data(tm, cf), 
                                                    index = ['Gait Detection', 
                                                             'Activity Level', 
                                                             'Tremor Score']))
    t_elapsed = ((time.time() - t1)/60.0)
    print(t_elapsed.__str__() + " mins", flush = True)
    return pd.concat([wa, df], axis = 1)

In [13]:
#for file in files:
def merge_file(file):
    t1 = time.time()
    print("1:")
    print(file)
    cf = pd.read_csv(calculated_features + "\\" + file,dtype={'measurement_id': int, 'value': float})
    wa = pd.read_csv(watch_accelerometer + "\\" + file,dtype={'user_id':int,'x':float,'y':float,'z':float})

    if os.path.isfile(combined_data + "\\" + file[0:-3] + 'pkl'):
        t_elapsed = ((time.time() - t1)/60.0)
        print(t_elapsed.__str__() + " mins")
    #     return

    wa['timestamp'] = pd.to_datetime(wa['timestamp'])
    wa['timestamp2'] = [(tm - datetime.timedelta(minutes=0,
                                                 seconds=tm.second % 5,
                                                 microseconds=tm.microsecond)) for tm in wa.timestamp]

    Gait = cf[cf.measurement_id==13][['value','timestamp']]
    ActLevel = cf[cf.measurement_id==10][['value','timestamp']]
    TremorScore = cf[cf.measurement_id==7][['value','timestamp']]

    Gait = Gait.rename(columns={'value':'Gait'})
    ActLevel = ActLevel.rename(columns={'value':'Activity Level'})
    TremorScore = TremorScore.rename(columns={'value':'TremorScore'})

    Gait = pd.merge(Gait,ActLevel,on='timestamp')
    Gait = pd.merge(Gait,TremorScore, on='timestamp')
    Gait.timestamp = pd.to_datetime(Gait.timestamp)
    wa = pd.merge(wa,Gait,left_on='timestamp2',right_on='timestamp')
    wa = wa.drop(columns=['timestamp2','timestamp_y'], axis=1)
    wa = wa.rename(columns={'timestamp_x':'timestamp'})

    with open(combined_data + "\\" + file[0:-3] + 'pkl','wb') as savefile:
        pickle.dump(wa, savefile)
    #     wa.to_csv(combined_data + "\\" + file)

    t_elapsed = ((time.time() - t1)/60.0)
    print(t_elapsed.__str__() + " mins")

In [9]:
with open(combined_data + "\\" +'142578 2017-09-18.pkl','rb') as savefile:
    D = pickle.load(savefile)

In [22]:
for file in files:
    merge_file(file)
#     df = merge_file_2(file)
#     print(df)  
# df.style

1:
142578 2017-09-18.csv
0.01420141855875651 mins
1:
142616 2018-02-05.csv
0.010151012738545736 mins
1:
142557 2017-06-29.csv
0.025035834312438963 mins
1:
142616 2017-10-03.csv
0.03332000176111857 mins
1:
142620 2017-10-30.csv
0.036453644434611 mins
1:
142568 2017-09-18.csv


KeyboardInterrupt: 

In [14]:
#file = files[0]
#cf = pd.read_csv(calculated_features + "\\" + file)
#wa = pd.read_csv(watch_accelerometer + "\\" + file)

#wa['timestamp'] = pd.to_datetime(wa['timestamp'])
#cf['timestamp'] = pd.to_datetime(cf['timestamp'])

#time = wa.loc[1, 'timestamp']
#tm = datetime.fromtimestamp(time)
#res = find_corresponding_cf_data(time, cf)
#for item in res:
#    print(item)
#print (res)

0.0
0.08473824382398823
0.0870042880649327
[0.0, 0.08473824382398823, 0.0870042880649327]


In [15]:
pool = ThreadPool()
pool.map(merge_file, files)
pool.close()
pool.join()

1:
142578 2017-09-18.csv
1:
142620 2017-10-30.csv
1:1:1:
142558 2017-09-21.csv
1:1:
142593 2017-11-06.csv
1:
142612 2017-09-06.csv

142604 2017-10-02.csv

142580 2017-08-01.csv

142608 2017-09-11.csv


C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\threading.py:864: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


0.05462212959925334 mins
0.3960729400316874 mins
1:
142616 2018-02-05.csv
0.4659465909004211 mins
1:
142557 2017-06-29.csv
1.2467579960823059 mins
1:
142568 2017-09-18.csv
1.0471213698387145 mins
1:
142616 2017-10-03.csv
1.2497082948684692 mins
1:
142616 2017-08-01.csv
4.108910846710205 mins
1:
142568 2018-02-28.csv
4.789378889401754 mins
1:
142601 2017-07-13.csv
5.290895700454712 mins
1:
142577 2017-11-30.csv
5.697753047943115 mins
1:
142595 2017-10-12.csv
5.157982416947683 mins
1:
142558 2017-07-14.csv
3.5823248624801636 mins
1:
142623 2017-10-12.csv
8.050471635659536 mins
1:
142623 2017-08-03.csv
8.658415750662487 mins
1:
142619 2017-08-10.csv
6.640280624230702 mins
1:
142615 2017-10-05.csv
8.41002113421758 mins
1:
142621 2017-10-23.csv
11.366129493713379 mins
1:
142603 2017-10-13.csv
10.917536262671153 mins
1:
142609 2017-09-01.csv
13.726350061098735 mins
1:
142603 2017-08-07.csv
11.427241643269857 mins
1:
142585 2017-09-20.csv
13.621263917287191 mins
1:
142604 2017-08-14.csv
13.11

C:\Users\nshawen\AppData\Local\Continuum\anaconda3\lib\threading.py:864: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)


9.031059177716573 mins
1:
142598 2017-07-10.csv
2.333834771315257 mins
1:
142605 2017-08-02.csv
4.560702717304229 mins
1:
142622 2017-08-17.csv
10.729617750644683 mins
1:
142619 2018-01-22.csv
11.259537796179453 mins
1:
142563 2017-07-18.csv
6.451664781570434 mins
1:
142578 2017-09-05.csv
7.841017059485117 mins
1:
142562 2017-07-07.csv
4.571890195210774 mins
1:
142617 2017-08-01.csv
4.351089203357697 mins
1:
142623 2017-08-17.csv
5.844385349750519 mins
1:
142618 2017-10-30.csv
7.6282332619031274 mins
1:
142578 2018-02-19.csv
7.734523542722067 mins
1:
142606 2017-10-13.csv
6.267134634653727 mins
1:
142580 2017-07-14.csv
6.103228227297465 mins
1:
142567 2018-01-11.csv
9.258781357606251 mins
1:
142600 2017-07-11.csv
7.249797979990642 mins
1:
142585 2018-02-02.csv
4.268668536345164 mins
1:
142617 2017-10-03.csv
11.997862573464712 mins
1:
142593 2018-01-29.csv
6.040656244754791 mins
1:
142600 2017-07-25.csv
2.843091670672099 mins
1:
142558 2017-06-29.csv
4.911577200889587 mins
1:
142568 201

In [12]:
pool = ThreadPool()

In [15]:
pool._processes

8